<a href="https://colab.research.google.com/github/revirevy/KAGGLE/blob/main/Nix_COMFY_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ps fax | grep comfy

    873 ?        S      0:00  |       \_ /bin/bash -c ps fax | grep comfy
    875 ?        S      0:00  |           \_ grep comfy


In [11]:
import subprocess
import os, sys
import subprocess
import time
import zipfile
import shutil
import requests
from tqdm import tqdm  # For progress tracking


from datetime import datetime
import threading
from IPython.core.getipython import get_ipython

In [7]:
def install_package(url):
    """Install a package from a given URL."""
    try:
        print("="*60, f"Starting to install package from {url}...", "-"*60, sep="\n")
        package_name = url.split('/')[-1].replace('.git', '')
        package_path = f"/content/ComfyUI/custom_nodes/{package_name}"

        if not os.path.exists("/content/ComfyUI/custom_nodes/"):
            os.makedirs("/content/ComfyUI/custom_nodes/")

        os.chdir("/content/ComfyUI/custom_nodes")

        if not os.path.exists(package_path):
            subprocess.run(['git', 'clone', url, '--recursive'], check=True, text=True, capture_output=True)
            print(f"Package {url} path newly git cloned")
        else:
            os.chdir(package_path)
            subprocess.run(['git', 'pull', '--all'], check=True, text=True, capture_output=True)
            print(f"Package {url} path exist already, just updated")

        if os.path.exists("requirements.txt"):
            subprocess.run(['uv','pip', 'install', '--system', '-r', 'requirements.txt', '--quiet'], check=True, text=True, capture_output=True)

        print("="*60, f"Package from {url} installed successfully.", "-"*60, sep="\n")
    except Exception as e:
        print(f"An error occurred while installing package from {url}: {str(e)}")
        # You can add additional error handling or logging here if needed

In [8]:
def setup_comfyui(install_path="/content"):  # Default to /content for Colab
    """Set up ComfyUI and install necessary packages."""
    print("="*60, "Starting to set up ComfyUI...", "-"*60, sep="\n")
    os.chdir(install_path)
    comfyui_path = os.path.join(install_path, "ComfyUI")
    if not os.path.exists(comfyui_path) or not os.path.exists(os.path.join(comfyui_path, "requirements.txt")):
        subprocess.run(['git', 'clone', 'https://github.com/comfyanonymous/ComfyUI.git', '--recursive'], check=True, text=True, capture_output=True, cwd=install_path)
    os.chdir(comfyui_path)
    subprocess.run(['git', 'pull', '--all'], cwd=comfyui_path)
    subprocess.run(['uv', 'pip', 'install', '--system', '-r', 'requirements.txt', '--quiet'], cwd=comfyui_path)

    model_dirs = [
        os.path.join(comfyui_path, "models/LLM"),
        os.path.join(comfyui_path, "models/pulid"),
        os.path.join(comfyui_path, "models/clip_vision"),
        os.path.join(comfyui_path, "models/xlabs/contronets/"),
        os.path.join(comfyui_path, "models/InstantIR/models/")
    ]
    for dir_path in model_dirs:
        os.makedirs(dir_path, exist_ok=True)
    print("="*60, "ComfyUI set up successfully.", "-"*60, sep="\n")

In [9]:
def start_comfyui_instances():
    """Start ComfyUI instances."""
    print("="*60, "Starting ComfyUI instances...", "-"*60, sep="\n")
    os.chdir("/content/ComfyUI/")  # Changed to /content/ComfyUI/
    xP1 = subprocess.Popen([sys.executable, "main.py", "--cuda-device", "0", "--port", "8188", "--fp8_e4m3fn-text-enc","--fp8_e4m3fn-unet", "--highvram"]) #--fp8_e4m3fn-text-enc --fp8_e4m3fn-unet
    time.sleep(10)
    xP2 = subprocess.Popen([sys.executable, "main.py", "--cuda-device", "1", "--port", "8189", "--fp8_e4m3fn-text-enc","--fp8_e4m3fn-unet", "--highvram"]) #, "--highvram"])
    time.sleep(10)
    print("="*60, "ComfyUI instances started successfully.", "-"*60, sep="\n")

def start_playit_agent():
    """Start the Playit agent."""
    print("="*60, "Starting Playit agent...", "-"*60, sep="\n")
    os.chdir("/content/ComfyUI/")  # Changed to /content/ComfyUI/
    if not os.path.exists('/content/ComfyUI/playit-linux-amd64'):  # Changed path
        subprocess.run(['wget', 'https://github.com/playit-cloud/playit-agent/releases/download/v0.15.26/playit-linux-amd64'])
    subprocess.run(['chmod', '+x', './playit-linux-amd64'])
    os.makedirs("/content/ComfyUI/logs", exist_ok=True)  # Changed path
    subprocess.Popen("nohup /content/ComfyUI/playit-linux-amd64 >> /content/ComfyUI/logs/playit.log 2>&1 &", bufsize=0, shell=True) # Changed path
    time.sleep(10)
    xresult = subprocess.run(['tail', '-n', '20', '/content/ComfyUI/logs/playit.log'] ,capture_output=True,text=True) # Changed path
    print(xresult.stdout,xresult.stderr,sep='\n'+'.'*10)
    print("","="*60, "Playit agent started successfully.", "-"*60, sep="\n")

In [ ]:
def main():
    """Main function to orchestrate the setup and execution."""
    print("="*60, "Starting main function...", "-"*60, sep="\n")
    # Install uv
    print("="*60, "Installing uv...", "-"*60, sep="\n")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-U', 'pip', 'uv', '-q'], check=True, text=True, capture_output=True)
    # subprocess.run(['uv','pip', 'install', '--system', 'sageattention', '--quiet'])


    # Setup ComfyUI
    setup_comfyui()

        # Setup ComfyUI
    setup_comfyui()

    # Install packages
    packages = [
        "https://github.com/comfyanonymous/ComfyUI_bitsandbytes_NF4",
        "https://github.com/WASasquatch/was-node-suite-comfyui",
        "https://github.com/rgthree/rgthree-comfy",
        "https://codeberg.org/Gourieff/comfyui-reactor-node",
        "https://github.com/BlenderNeko/ComfyUI_Noise",
        "https://github.com/ltdrdata/ComfyUI-Impact-Pack",
        "https://github.com/ltdrdata/ComfyUI-Impact-Subpack",

        "https://github.com/yolain/ComfyUI-Easy-Use",
        "https://github.com/kijai/ComfyUI-KJNodes",
        "https://github.com/kijai/ComfyUI-Florence2",
        "https://github.com/XLabs-AI/x-flux-comfyui",
        "https://github.com/crystian/ComfyUI-Crystools",
        "https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes",
        "https://github.com/city96/ComfyUI-GGUF",
        "https://github.com/cubiq/PuLID_ComfyUI",
        "https://github.com/cubiq/ComfyUI_InstantID",
        "https://github.com/cubiq/ComfyUI_IPAdapter_plus",
        "https://github.com/comfyanonymous/ComfyUI_experiments",
        "https://github.com/chengzeyi/Comfy-WaveSpeed",
        "https://github.com/Fannovel16/comfyui_controlnet_aux",
        "https://github.com/ShmuelRonen/ComfyUI-FreeMemory",
        "https://github.com/jamesWalker55/comfyui-various",
        "https://github.com/kijai/ComfyUI-SUPIR",
        "https://github.com/klinter007/klinter_nodes",
        "https://github.com/holchan/ComfyUI-ModelDownloader",
        "https://github.com/KoreTeknology/ComfyUI-Universal-Styler",
        "https://github.com/marduk191/ComfyUI-Fluxpromptenhancer",
        "https://github.com/fairy-root/Flux-Prompt-Generator",
        "https://github.com/VykosX/ControlFlowUtils",
        "https://github.com/godmt/ComfyUI-List-Utils",
        "https://github.com/pythongosssss/ComfyUI-WD14-Tagger",
        "https://github.com/miaoshouai/ComfyUI-Miaoshouai-Tagger",
        "https://github.com/GraftingRayman/ComfyUI_GraftingRayman",
        "https://github.com/pollockjj/ComfyUI-MultiGPU",
        "https://github.com/dchatel/comfyui_facetools",
        "https://github.com/fssorc/ComfyUI_FaceShaper",

        "https://github.com/vuongminh1907/ComfyUI_ZenID",
        "https://github.com/nuanarchy/ComfyUI-NuA-FlashFace",
        "https://github.com/nosiu/comfyui-instantId-faceswap",
        "https://github.com/EnragedAntelope/ComfyUI-EACloudNodes",
        "https://github.com/nicofdga/DZ-FaceDetailer",
        "https://github.com/mav-rik/facerestore_cf",
        "https://github.com/smthemex/ComfyUI_InstantIR_Wrapper",
        "https://github.com/cubiq/ComfyUI_essentials",


        "https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet",
        "https://github.com/EvilBT/ComfyUI_SLK_joy_caption_two",
        "https://github.com/PowerHouseMan/ComfyUI-AdvancedLivePortrait",
        "https://github.com/ssitu/ComfyUI_UltimateSDUpscale",
        "https://github.com/kijai/ComfyUI-CCSR",
        "https://github.com/TheBill2001/comfyui-upscale-by-model",
        "https://github.com/zentrocdot/ComfyUI-RealESRGAN_Upscaler",
        "https://github.com/traugdor/ComfyUI-UltimateSDUpscale-GGUF",
        "https://github.com/jags111/efficiency-nodes-comfyui",
        "https://github.com/kijai/ComfyUI-segment-anything-2",
        "https://github.com/Extraltodeus/ComfyUI-AutomaticCFG",
        "https://github.com/ltdrdata/ComfyUI-Inspire-Pack",
        "https://github.com/evanspearman/ComfyMath",
        "https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes",
        "https://github.com/storyicon/comfyui_segment_anything",


        "https://github.com/zhangp365/ComfyUI-utils-nodes",
        "https://github.com/pythongosssss/ComfyUI-Custom-Scripts",
        "https://github.com/revirevy/Comfyui_saveimage_imgbb",
        "https://github.com/ltdrdata/ComfyUI-Manager",

        ]
    for package in packages:
        install_package(package)

    # Setup ComfyUI
    setup_comfyui()

    # Link models
    # link_models()

    # link_additional_models
    # link_additional_models()

    # Start ComfyUI instances
    start_comfyui_instances()

    # Start Playit agent
    start_playit_agent()

    # download antelope
    # down_antelope()

    # down_landmark()

    # Start scheduler
    # scheduler_thread = threading.Thread(target=scheduler)
    # scheduler_thread.daemon = True
    # scheduler_thread.start()

    # print("="*60, "Scheduler started in background. It will execute the first cell every 1 minute.", "-"*60, sep="\n")

if __name__ == "__main__":
    main()

Starting main function...
------------------------------------------------------------
Installing uv...
------------------------------------------------------------
Starting to set up ComfyUI...
------------------------------------------------------------
ComfyUI set up successfully.
------------------------------------------------------------
Starting to set up ComfyUI...
------------------------------------------------------------
ComfyUI set up successfully.
------------------------------------------------------------
Starting to install package from https://github.com/comfyanonymous/ComfyUI_bitsandbytes_NF4...
------------------------------------------------------------
Package https://github.com/comfyanonymous/ComfyUI_bitsandbytes_NF4 path newly git cloned
Package from https://github.com/comfyanonymous/ComfyUI_bitsandbytes_NF4 installed successfully.
------------------------------------------------------------
Starting to install package from https://github.com/WASasquatch/was-nod